In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [2]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_19616\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
url = "https://mi.tv/br/entrar"
driver.get(url)
driver.window_handles;
driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.CONTROL+"t");
driver.find_element(By.XPATH, '//*[@id="login-email"]').send_keys("osumorog@gmail.com")
driver.find_element(By.XPATH, '//*[@id="login-password"]').send_keys("firefire")
driver.find_element(By.XPATH, '//*[@id="btn-email-login-perform"]').click()

In [8]:
driver.get("https://mi.tv/br/perfil/selecionar-canais")
page_source =driver.page_source
soup = BeautifulSoup(page_source,'html.parser')

In [10]:
buttons = soup.find("div", {"class":"container"})
buttons = buttons.find_all("span",{"class":"channel-name"})
channel_names = []
for button in buttons:
    channel_names.append(button.text.replace('\n',""))
    
for i in range(1, len(channel_names[1:])):
    channel_names[i] = channel_names[i] .replace("+", "")
    channel_names[i] = channel_names[i] .replace("&", "_")
    channel_names[i] = channel_names[i] .replace("-", "") 
print(len(channel_names))

825


In [12]:
search_urls = []
links = []
base_url ="https://mi.tv"
for i in channel_names[1:]:
    search_link = 'https://mi.tv/br/pesquisar/'+i
    search_urls.append(search_link)
    
print(len(search_urls))

for i in search_urls:    
    driver.get(i)
    page_source =driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    channel_links = soup.find("ul", {"class":"channels"})
    if channel_links !=None:
        channel_links = channel_links.find_all("li")
        for channel_link in channel_links:
            tv_link = channel_link.find("a").get("href")
            tv_link = base_url+tv_link
            if tv_link not in links:
                links.append(tv_link)

824


slink = "https://mi.tv/br/canais/amc-hd/2022-09-01"
driver.get(slink)
page_source =driver.page_source

class Data:
    def __init__(self, link, date):
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.link = link+"/"+date

    def get_data(self):
        from datetime import datetime
        self.driver.get(self.link)
        page_source =self.driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        try:
            channel_detail = soup.find("div", {"class":"channel-info"})
        except:
            channel_detail = None
        try:
            channel_name = channel_detail.find("h1").text.replace('\n',"")
        except:
            channel_name = None
        try:
            program_date = channel_detail.find("span").text.replace('\n',"")
        except:
            program_date = None
        try:
            programs = soup.find("ul", {"class":"broadcasts time24"})
        except:
            programs  =None
        try:
            programs = programs.find_all("li")
        except:
            programs  =None
        for program in programs:
            content = program.find("div", {"class":"content"})
            try:
                prog_link = program.find("a", {"class":"program-link"}).get("href")   
            except:
                prog_link = None
            try:
                prog_name = content.find("h2").text.replace('\n',"")
                prog_name = re.sub(r"[\t]*", "", prog_name)
            except:
                prog_name = None
                
            try:
                prog_time = content.find("span", {"class":"time"}).text.replace('\n',"")
            except:
                prog_time = None
            try:
                prog_genre = content.find("span", {"class":"sub-title"}).text.replace('\n',"")
                prog_genre = re.sub(r"[\t]*", "", prog_genre)
            except:
                prog_genre = None
            try:
                prog_synopsis = content.find("p", {"class":"synopsis"}).text.replace('\n',"")
                prog_synopsis = re.sub(r"[\t]*", "", prog_synopsis)
            except:
                prog_synopsis = None
                
            try:
                now = datetime.now()
                date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
            except:
                date_scraped = None
            
            tv = {"channel":channel_name, "program name":prog_name, "program time":prog_time,"program genre":prog_genre, "program synopsis":prog_synopsis, "program link":prog_link,  "date":program_date, "scraped link":self.link, "scrape date":date_scraped}
            
            return tv
    

In [15]:
from datetime import datetime
data = []
for link in links:
    driver.get(link)
    page_source =driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    
    programs = soup.find("ul", {"class":"broadcasts time24"})
    if programs != None:
        programs = programs.find_all("li")
        for program in programs:
            content = program.find("div", {"class":"content"})
            channel_detail = soup.find("div", {"class":"channel-info"})
            try:
                channel_name = channel_detail.find("h1").text.replace('\n',"")
            except:
                channel_name = None
            try:
                program_date = channel_detail.find("span").text.replace('\n',"")
            except:
                program_date = None
            try:
                prog_link = program.find("a", {"class":"program-link"}).get("href")   
            except:
                prog_link = None
            try:
                prog_name = content.find("h2").text.replace('\n',"")
                prog_name = re.sub(r"[\t]*", "", prog_name)
            except:
                prog_name = None
                        
            try:
                prog_time = content.find("span", {"class":"time"}).text.replace('\n',"")
            except:
                prog_time = None
            try:
                prog_genre = content.find("span", {"class":"sub-title"}).text.replace('\n',"")
                prog_genre = re.sub(r"[\t]*", "", prog_genre)
            except:
                prog_genre = None
            try:
                prog_synopsis = content.find("p", {"class":"synopsis"}).text.replace('\n',"")
                prog_synopsis = re.sub(r"[\t]*", "", prog_synopsis)
            except:
                prog_synopsis = None
                        
            try:
                now = datetime.now()
                date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
            except:
                date_scraped = None
                    
            tv = {"channel":channel_name, "program name":prog_name, "program time":prog_time,"program genre":prog_genre, "program synopsis":prog_synopsis, "program link":prog_link,  "date":program_date, "scraped link":link, "scrape date":date_scraped}
                    
            data.append(tv)
    

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
df = pd.DataFrame.from_dict(data)
df